In [ ]:
!pip install datasets transformers
! pip install -U accelerate
! pip install -U transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer
import json

In [ ]:
model_checkpoint = "hfl/chinese-bert-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
##########
# 1. Load the data
##########
# Import the training data
with open('/content/drive/My Drive/NQuAD/NQuAD_train_first_10k_saved.json', 'r', encoding='utf-8') as file:
        data_train = json.load(file)

# Import the testing data
with open('/content/drive/My Drive/NQuAD/NQuAD_test_first_2k_saved.json', 'r', encoding='utf-8') as file:
        data_test = json.load(file)

In [ ]:
data_train[0]

In [ ]:
data_test[0]

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value

# 定义数据集特征，为每个answer提供单独的字段
features = Features({
    'context': Value('string'),
    'answer0': Value('string'),
    'answer1': Value('string'),
    'answer2': Value('string'),
    'answer3': Value('string'),
    'label': ClassLabel(num_classes=4, names=['0', '1', '2', '3'])
})

# 准备数据
prepared_train_data = {
    'context': [],
    'answer0': [],
    'answer1': [],
    'answer2': [],
    'answer3': [],
    'label': []
}

prepared_test_data = {
    'context': [],
    'answer0': [],
    'answer1': [],
    'answer2': [],
    'answer3': [],
    'label': []
}

for item in data_train:
    prepared_train_data['context'].append(item['context'].strip())
    prepared_train_data['label'].append(item['ans'])
    for i in range(4):
        prepared_train_data[f'answer{i}'].append(item[f'answer{i}'].strip())


for item in data_test:
    prepared_test_data['context'].append(item['context'].strip())
    prepared_test_data['label'].append(item['ans'])
    for i in range(4):
        prepared_test_data[f'answer{i}'].append(item[f'answer{i}'].strip())

# 创建单个 Dataset
train_data = Dataset.from_dict(prepared_train_data, features=features)
test_data = Dataset.from_dict(prepared_test_data, features=features)

In [ ]:
datasets = DatasetDict({
    'train': train_data,
    'test': test_data
})

In [ ]:
datasets['train'][0]

In [ ]:
datasets['train'][1]

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

In [ ]:
def show_one(example):
    print(f"Context: {example['context']}")
    print(f"  A - {example['answer0']}")
    print(f"  B - {example['answer1']}")
    print(f"  C - {example['answer2']}")
    print(f"  D - {example['answer3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

show_one(datasets["train"][0])

# Context: 宏達電（2498）昨(21)日股東會上，董事長王雪紅應允小股東請求，自掏腰包贈送股東會議程結束前完成報到程序且人在會場的小股東（不含公司內部股東），每人一台宏達電旗艦手機新HTC One 32G手機、單機市價約2.19萬元，創下台灣股東會史上單價最高的贈禮。同時，宏達電董監改選，前台積電（2330）總經理蔡力行當選新任董事，被外界認為是跟台積電聯手抗韓，針對三星電子的科技競爭而進行的結盟。據統計，當日符合條件股東數約100多位，依單機2.19萬元計算，王雪紅大手筆送給在場關心宏達電營運的小股東們逾200多萬元大禮。經過各家媒體近兩日報導，「宏達電」、「HTC」、「新HTC One」在網路上討論的熱烈程度又升高了，達到品牌與知名度的提升效果，宏達電行銷功力可謂更上層樓。宏達電今年的股東會也全面進行董監事改選，當選董事名單包括王雪紅、陳文琦、卓火土、蔡力行、David Bruce Yoffie；獨立董事為林振國、Josef Felder；新任監察人為威智投資(股)公司、朱黃傑。除了蔡力行是新赴任的宏達電董事外，其餘本次當選董事、監察人都為續任。蔡力行是台積電（2330）前總經理、現任台積電太陽能與固態照明董事長；因此外界也多解讀，在台積電董事長張忠謀公開多次盛讚宏達電手機產品，並倡議聯發科（2454）、鴻海（2317）、台積電與宏達電在各自的半導體IC、面板、晶圓產業、消費性手機領域一同對付三星電子；蔡力行昨日獲選出任宏達電董事一職，可謂是台灣科技廠聯手抗韓的最新發展。
#   A - 王雪紅贈股東100G新One，創台股東會單價最高贈品
#   B - 王雪紅贈股東2.19G新One，創台股東會單價最高贈品
#   C - 王雪紅贈股東21G新One，創台股東會單價最高贈品
#   D - 王雪紅贈股東32G新One，創台股東會單價最高贈品

# Ground truth: option D

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "hfl/chinese-bert-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, padding=True, truncation=True, max_length=512)

In [ ]:
print(datasets['train'][0]['answer0'])

In [ ]:
ans_names = ['answer0', 'answer1', 'answer2', 'answer3']

def preprocess_function(examples):

    # 预处理输入tokenizer的输入
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["context"]]# 构造和备选项个数相同的问题句，也是tokenizer的第一个句子

    # 构造所有的second sentences，每个second sentence是一个备选答案
    second_sentences = [
        [examples[ans][i] for ans in ans_names] for i in range(len(examples["context"]))
    ]

    # # Flatten everything
    first_sentences = sum(first_sentences, []) # 合并成一个列表方便tokenizer一次性处理
    second_sentences = sum(second_sentences, [])# 合并成一个列表方便tokenizer一次性处理

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True,  max_length=512)

    # Un-flatten
    # 转化成每个样本（一个样本中包括了四个k=[问题1,问题1,问题1,问题1],v=[备选项1,备选项2,备选项3,备选项4]）

    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
examples = datasets["train"][:2]
features = preprocess_function(examples)

In [ ]:
idx = 0
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

In [ ]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForMultipleChoice
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

In [ ]:
import accelerate
print(accelerate.__version__)

In [ ]:
task='ner'
batch_size = 5

from transformers import  TrainingArguments

In [ ]:
args = TrainingArguments(
    "my_model_output",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #features:[{'attention_mask':[[],[],...],'input_ids':[[],[],...,'label':_},{'attention_mask':[[],[],...],'input_ids':[[],[],...,'label':_}]
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features] #将label单独弹出，features:[{'attention_mask':[[],[],...],'input_ids':[[],[],...]},{'attention_mask':[[],[],...],'input_ids':[[],[],...]}]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])

        #feature:{'attention_mask':[[],[],...],'input_ids':[[],[],...]}
        #flattened_features:[[{'attention_mask':[],'input_ids':[]},{},{},{}],[]....]
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        #flattened_features:[{'attention_mask':[],'input_ids':[]},{},{},{},{}....]
        flattened_features = sum(flattened_features, [])

        # batch: {'attention_mask':[[],[],[],[],[],[],...],'input_ids':[[],[],[],[],[],[],...]}
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        # batch: {'attention_mask':[[[],[],[],[]],[[],[],[],[]],[...],...],'input_ids':[[[],[],[],[]],[[],[],[],[]],[...],...]}
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        # batch: {'attention_mask':[[[],[],[],[]],[[],[],[],[]],[...],...],'input_ids':[[[],[],[],[]],[[],[],[],[]],[...],...],'label':[]}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in tokenized_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
features[0]

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

In [ ]:
show_one(datasets["train"][8])
#    Context: Someone walks over to the radio.
#      A - Someone hands her another phone.
#      B - Someone takes the drink, then holds it.
#      C - Someone looks off then looks at someone.
#      D - Someone stares blearily down at the floor.
#
#    Ground truth: option D

In [ ]:
import numpy as np
from datasets import load_metric
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
from transformers import  Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
!export TF_FORCE_GPU_ALLOW_GROWTH=true

In [ ]:
trainer.train()